In [1]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
import requests
from selenium.webdriver.support.ui import WebDriverWait
import selenium.webdriver.support.expected_conditions as EC
from tqdm.notebook import tqdm
import time
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
import time
from tqdm import tqdm

In [2]:
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument("--incognito")
driver = webdriver.Chrome(executable_path = r'chromedriver.exe',chrome_options = chrome_options)

C:\Users\mohit\AppData\Local\Temp\ipykernel_12456\472493040.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path = r'chromedriver.exe',chrome_options = chrome_options)
C:\Users\mohit\AppData\Local\Temp\ipykernel_12456\472493040.py:3: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome(executable_path = r'chromedriver.exe',chrome_options = chrome_options)


In [3]:
inputlist = pd.read_excel(r"coolblue input list.xlsx")
inputlist

,Asin,url
0,926618,https://www.coolblue.nl/en/product/926618/hp-p...
1,928607,https://www.coolblue.nl/en/product/928607/hp-p...
2,926603,https://www.coolblue.nl/en/product/926603/hp-p...
3,907753,https://www.coolblue.nl/en/product/907753/hp-p...
4,912577,https://www.coolblue.nl/en/product/912577/hp-p...
5,907759,https://www.coolblue.nl/en/product/907759/hp-p...
6,928607,https://www.coolblue.nl/en/product/928607/hp-p...
7,928271,https://www.coolblue.nl/en/product/928271/hp-p...
8,926610,https://www.coolblue.nl/en/product/926610/hp-p...
9,926542,https://www.coolblue.nl/en/product/926542/hp-p...


In [4]:
inputlist.drop_duplicates(inplace=True)
inputlist.dropna(inplace = True)
asinlist = list(inputlist['Asin'].unique())
asinlist

[926618,
 928607,
 926603,
 907753,
 912577,
 907759,
 928271,
 926610,
 926542,
 926546]

In [5]:
df = pd.DataFrame(columns=['Product ID','Product Title','Product URL','Image URL','No. of Images',
                           'MRP','Price', 'Specifications', 'Description', 'Pros', 'Cons', 'Average Rating', 'Review Count'])

In [6]:
for val in tqdm(range(0,10)):
    
    i=asinlist[val]
    print(i)
    coolblue_url = "https://www.coolblue.nl/en/product/{}".format(i)
    driver.get(coolblue_url)
    print(coolblue_url)
    driver.execute_script("window.scrollTo(0,500)")
    try:
        #Final for Images# Locate the button that expands additional images
        imgbutton = driver.find_element(By.XPATH, "/html/body/main/div[3]/div[2]/div[3]/div[1]/div/div[1]/div/div/div[2]/ul/li[8]/button")
        time.sleep(5)
        imgbutton.click()  # Click the button to load additional images
    except:
        print('no button')
        pass
    time.sleep(1)
    soup=BeautifulSoup(driver.page_source,"html.parser")
    productcomplete=soup.find_all("div",class_="pdp-block")
    try:
        prod=soup.find("div",class_="product-page")
        product_title=prod.find("h1",class_="js-product-name").text.strip()
        print("Product Title:", product_title)
    except AttributeError:
        product_title=""
        print("Product title not found.")
    
    try:
        imgsrc = driver.find_elements(By.XPATH,'//ul[@class="product-media-gallery-thumbnails__wrapper js-media-gallery__thumbnails"]/li')
        imgcount = len(imgsrc)
        print("Image count:", imgcount)
        img_urls = []  # Initialize a list to store image URLs
        for j in range(len(imgsrc)):
            img_url = imgsrc[j].find_element(By.XPATH, './img').get_attribute('src')
            img_urls.append(img_url)
            print("Image URL", j + 1, ":", img_url)
    except NoSuchElementException:
        print("No image found:")
        img_urls = []

    
    try:
        prices=soup.find("div",class_="js-sales-price-wrapper")
        mrp=prices.find("span",class_='sales-price__former-price').text.strip()
        cleaned_mrp = mrp.replace(",", "").replace("-", "")
        print("MRP :",cleaned_mrp)
    except:
        cleaned_mrp=''
        print("No MRP found")
        
    try:
        prices=soup.find("div",class_="js-sales-price-wrapper")
        price=prices.find("strong",class_='sales-price__current js-sales-price-current').text.strip()
        cleaned_price = price.replace(",", "").replace("-", "")
        print("Price :",cleaned_price)
    except:
        cleaned_price=''
        print("No Price found")
        
    # Product specifications
    spec_url = coolblue_url + "#product-specifications"
    driver.get(spec_url)
    soup_specs = BeautifulSoup(driver.page_source, "html.parser")
    try:
        specs = soup_specs.find("div", class_="section--2 product-specs--skeleton js-specifications-content p--3 pt--0 p--0@sm")
        spec_items = specs.find_all('dl', class_='product-specs__list-item')
        specifications = {}

        for spec_item in spec_items:
            property_name_element = spec_item.find('span', class_='js-highlightable')
            property_value_element = spec_item.find('dd', class_='js-spec-value')

            if property_name_element and property_value_element:
                property_name = property_name_element.get_text(strip=True)
                property_value = property_value_element.get_text(strip=True)
                specifications[property_name] = property_value
                
    except:
        print("No Specifications :")
        specifications=[]
        
    # Product description and pros/cons
    desc_url = coolblue_url + "#product-information"
    driver.get(desc_url)
    soup_desc = BeautifulSoup(driver.page_source, "html.parser")
    try:
        description_sections = soup_desc.find("div", class_="cms-content")
        description = description_sections.find("p").text
        print("Description :",description)
    except:
        print("No Description Found")
        description = []
    try:
        pros_cons1 = soup_desc.find("div", class_="col--12 col--9@sm")
        pros_cons = pros_cons1.find_all("span", class_="icon-with-text")
        pros = []
        cons = []
        for icon_text in pros_cons:
            icon = icon_text.find('span', class_='icon-with-text__icon')
            text = icon_text.find('span', class_='icon-with-text__text').get_text(strip=True)

            if 'icon--color-green' in icon.find('svg')['class']:
                pros.append(text)
            else:
                cons.append(text)
                        # Print the extracted pros and cons
        print("Pros:")
        for p in pros:
            print(p)

        print("\nCons:")
        for c in cons:
            print(c)
            
    except:
        
        print("No Pros Found")
        print("No Cons Found")
        
        pros=[]
        cons=[]
    
    # Average rating and review count
    avg_rating_url = coolblue_url + "#product-reviews"
    driver.get(avg_rating_url)
    soup_rating = BeautifulSoup(driver.page_source, "html.parser")
    try:
        average_rating_element = soup_rating.find("span", class_='review-rating__reviews text--truncate')
        if average_rating_element:
            average_rating_text = average_rating_element.text.strip()
            average_rating_parts = average_rating_text.split('(')

            average_rating = average_rating_parts[0].strip()
            review_count = average_rating_parts[1].split(' ')[0]

            print("Average Rating:", average_rating)
            print("Review Count:", review_count)
        else:
            print("Average rating element not found.")
            average_rating=[]
            review_count=[]
    except:
        print("Average rating element not found.")
        average_rating=[]
        review_count=[]
    

    # Append the extracted information as a new row to the DataFrame
    data = {'Product ID':i,'Product Title':product_title,'Product URL': coolblue_url,'Image URL':img_urls,'No. of Images':imgcount,
            'MRP': cleaned_mrp ,'Price': cleaned_price,'Specifications': specifications, 'Description': description, 'Pros': pros, 'Cons': cons,
            'Average Rating': average_rating, 'Review Count': review_count}
    df = df.append(data, ignore_index=True)
    time.sleep(2)

    # Print the extracted shop details
print(df)
print(val)

  0%|                                                                                           | 0/10 [00:00<?, ?it/s]

926618
https://www.coolblue.nl/en/product/926618
no button
Product Title: HP Pavilion 15-eg2951nd
Image count: 0
MRP : 699
Price : 599
Description : Edit photos, watch movies, and write emails with the 15-inch HP Pavilion 15-eg2951nd. The 12th generation Intel Core i5 processor and 8GB DDR4 RAM are powerful enough for that. Want to enjoy relaxing music while you work? The audio that comes from the Bang & Olufsen speakers sounds clearer than that of standard laptop speakers. Feel free to work at night, as the backlit keyboard shows you where to find the keys in the dark. Is your laptop empty, but you want to get back to work quickly? With HP Fast Charge, you can charge your laptop back up to 50% in about 45 minutes. That way, you can soon continue to work wirelessly. Store your photos, documents, and other files on the 512GB M.2 SSD. You don't have to worry about the security of your creations. That's because you can log in quick and easy with the fingerprint sensor. Find sustainability

C:\Users\mohit\AppData\Local\Temp\ipykernel_12456\2020724678.py:152: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(data, ignore_index=True)
 10%|████████▎                                                                          | 1/10 [00:24<03:38, 24.24s/it]

928607
https://www.coolblue.nl/en/product/928607
Product Title: HP Pavilion 14-dv2955nd
Image count: 10
Image URL 1 : https://image.coolblue.nl/75x75/products/1946319
Image URL 2 : https://image.coolblue.nl/75x75/products/1946320
Image URL 3 : https://image.coolblue.nl/75x75/products/1946321
Image URL 4 : https://image.coolblue.nl/75x75/products/1946316
Image URL 5 : https://image.coolblue.nl/75x75/products/1946318
Image URL 6 : https://image.coolblue.nl/75x75/products/1946322
Image URL 7 : https://image.coolblue.nl/75x75/products/1946315
Image URL 8 : https://image.coolblue.nl/75x75/products/1946317
Image URL 9 : https://image.coolblue.nl/75x75/products/1946313
Image URL 10 : https://image.coolblue.nl/75x75/products/1946314
No MRP found
Price : 799
Description : Edit photos and multitask between light programs with the 14-inch HP Pavilion 14-dv2955nd. The 12th generation Intel Core i5 processor and 16GB DDR4 RAM are powerful enough for that. You can also easily take this HP with you, 

C:\Users\mohit\AppData\Local\Temp\ipykernel_12456\2020724678.py:152: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(data, ignore_index=True)
 20%|████████████████▌                                                                  | 2/10 [00:43<02:51, 21.50s/it]

...

926542
https://www.coolblue.nl/en/product/926542
Product Title: HP Pavilion x360 14-dy1957nd
Image count: 13
Image URL 1 : https://image.coolblue.nl/75x75/products/1945819
Image URL 2 : https://image.coolblue.nl/75x75/products/1945821
Image URL 3 : https://image.coolblue.nl/75x75/products/1945823
Image URL 4 : https://image.coolblue.nl/75x75/products/1945818
Image URL 5 : https://image.coolblue.nl/75x75/products/1945815
Image URL 6 : https://image.coolblue.nl/75x75/products/1945820
Image URL 7 : https://image.coolblue.nl/75x75/products/1945825
Image URL 8 : https://image.coolblue.nl/75x75/products/1945824
Image URL 9 : https://image.coolblue.nl/75x75/products/1945822
Image URL 10 : https://image.coolblue.nl/75x75/products/1945817
Image URL 11 : https://image.coolblue.nl/75x75/products/1945816
Image URL 12 : https://image.coolblue.nl/75x75/products/1945814
Image URL 13 : https://image.coolblue.nl/75x75/products/1945813
MRP : 749
Price : 699
No Description Found
No Pros Found
No Cons Fou

C:\Users\mohit\AppData\Local\Temp\ipykernel_12456\2020724678.py:152: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(data, ignore_index=True)
 90%|██████████████████████████████████████████████████████████████████████████▋        | 9/10 [03:24<00:22, 22.99s/it]

926546
https://www.coolblue.nl/en/product/926546
Product Title: HP Pavilion x360 14-ek0956nd
Image count: 11
Image URL 1 : https://image.coolblue.nl/75x75/products/1945862
Image URL 2 : https://image.coolblue.nl/75x75/products/1945863
Image URL 3 : https://image.coolblue.nl/75x75/products/1945864
Image URL 4 : https://image.coolblue.nl/75x75/products/1945858
Image URL 5 : https://image.coolblue.nl/75x75/products/1945857
Image URL 6 : https://image.coolblue.nl/75x75/products/1945865
Image URL 7 : https://image.coolblue.nl/75x75/products/1945859
Image URL 8 : https://image.coolblue.nl/75x75/products/1945860
Image URL 9 : https://image.coolblue.nl/75x75/products/1945861
Image URL 10 : https://image.coolblue.nl/75x75/products/1945855
Image URL 11 : https://image.coolblue.nl/75x75/products/1945856
No MRP found
Price : 1.299
No Description Found
Pros:


Cons:

Average Rating: 8,5/10
Review Count: 40


C:\Users\mohit\AppData\Local\Temp\ipykernel_12456\2020724678.py:152: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(data, ignore_index=True)
100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [03:47<00:00, 22.70s/it]

  Product ID                 Product Title  \
0     926618       HP Pavilion 15-eg2951nd   
1     928607       HP Pavilion 14-dv2955nd   
2     926603       HP Pavilion 15-eg2972nd   
3     907753       HP Pavilion 14-dv1905nd   
4     912577        HP Pavilion14-ec1977nd   
5     907759       HP Pavilion 15-eh1915nd   
6     928271       HP Pavilion 15-eh3955nd   
7     926610       HP Pavilion 15-eg2980nd   
8     926542  HP Pavilion x360 14-dy1957nd   
9     926546  HP Pavilion x360 14-ek0956nd   

                                 Product URL  \
0  https://www.coolblue.nl/en/product/926618   
1  https://www.coolblue.nl/en/product/928607   
2  https://www.coolblue.nl/en/product/926603   
3  https://www.coolblue.nl/en/product/907753   
4  https://www.coolblue.nl/en/product/912577   
5  https://www.coolblue.nl/en/product/907759   
6  https://www.coolblue.nl/en/product/928271   
7  https://www.coolblue.nl/en/product/926610   
8  https://www.coolblue.nl/en/product/926542   
9  https://ww

In [7]:
df.head()

,Product ID,Product Title,Product URL,Image URL,No. of Images,MRP,Price,Specifications,Description,Pros,Cons,Average Rating,Review Count
0,926618,HP Pavilion 15-eg2951nd,https://www.coolblue.nl/en/product/926618,[],0,699,599,"{'Product number': '926618', 'Manufacturer cod...","Edit photos, watch movies, and write emails wi...","[With an Intel Core i5 processor and 8GB RAM, ...",[A minimum of 16GB RAM is recommended for edit...,"8,8/10",40
1,928607,HP Pavilion 14-dv2955nd,https://www.coolblue.nl/en/product/928607,[https://image.coolblue.nl/75x75/products/1946...,10,,799,"{'Product number': '928607', 'Manufacturer cod...",Edit photos and multitask between light progra...,[With the Intel Core i5 processor and 16GB RAM...,"[If you edit videos, choose a laptop with at l...",[],[]
2,926603,HP Pavilion 15-eg2972nd,https://www.coolblue.nl/en/product/926603,[https://image.coolblue.nl/75x75/products/1941...,10,899,749,"{'Product number': '926603', 'Manufacturer cod...",[],[Met de Intel Core i7 processor en het 16 giga...,[For 2D and 3D design in software like AutoCAD...,"8,8/10",40
3,907753,HP Pavilion 14-dv1905nd,https://www.coolblue.nl/en/product/907753,[https://image.coolblue.nl/75x75/products/1781...,9,869,819,"{'Product number': '907753', 'Manufacturer cod...","With the 14-inch HP Pavilion 14-dv1905nd, you ...","[With HP Fast Charge, you can charge your lapt...","[If you edit a lot of videos, choose a laptop ...","8,3/10",3
4,912577,HP Pavilion14-ec1977nd,https://www.coolblue.nl/en/product/912577,[https://image.coolblue.nl/75x75/products/1814...,10,979,879,"{'Product number': '912577', 'Manufacturer cod...",The HP Pavilion 14-ec1977nd is a 14-inch lapto...,[You can charge your laptop back up to 50% in ...,"[To design 3D projects, you need an Intel Core...","9,0/10",7


In [8]:
df.to_excel(r'coolblueoutput1.xlsx')